In [1]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
import tifffile
from PIL import Image
import os
import numpy as np
import time
import re
from skimage import data, color
import scipy.ndimage as ndimage
import h5py
import scipy.io as spio
import scipy as sp

import sys
import seaborn
seaborn.set(font_scale=2)
seaborn.set_style('whitegrid')
clrs = seaborn.color_palette()
from multiprocessing.dummy import Pool 

sys.path.append('/home/yves/Documents/')
import twoptb as MP



In [ ]:
import numpy as np
import scipy.optimize as op

In [ ]:
# Laplace Inference -----------------------------------------------------------
def negLogPosteriorUnNorm(xbar, ybar, C_big, d_big, K_bigInv, xdim, ydim):
    xbar = np.ndarray.flatten(np.asarray(xbar))
    ybar = np.ndarray.flatten(np.asarray(ybar))
    T = int(len(d_big)/ydim)

    C_big = np.asarray(C_big)
    d_big = np.asarray(d_big)

    K_bigInv = np.asarray(K_bigInv)

    A = np.dot(C_big.T, xbar) + d_big
    Aexp = np.exp(A)

    L1 = np.dot(Aexp, np.ones(ydim*T))
    L2 = - np.dot(ybar, A.T)
    L3 = 0.5*np.dot(xbar,np.dot(K_bigInv,xbar))

    L = L1 + L2 + L3

    # pdb.set_trace()
    return L

def negLogPosteriorUnNorm_grad(xbar, ybar, C_big, d_big, K_bigInv, xdim, ydim):
    xbar = np.asarray(xbar)
    ybar = np.asarray(ybar)

    A = np.dot(C_big.T, xbar) + d_big
    A = np.float64(A)
    Aexp = np.exp(A)

    dL1 = np.dot(Aexp,C_big.T)
    dL2 = - np.dot(ybar, C_big.T)
    dL3 = np.dot(xbar, K_bigInv)

    dL = dL1 + dL2 + dL3

    return dL

def negLogPosteriorUnNorm_hess(xbar, ybar, C_big, d_big, K_bigInv, xdim, ydim):
    xbar = np.asarray(xbar)
    ybar = np.asarray(ybar)

    T = int(len(xbar)/xdim)

    A = np.dot(C_big.T, xbar) + d_big
    A = np.float64(A)

    Aexp = np.exp(A)
    Aexpdiagonal = sp.sparse.spdiags(Aexp,0,ydim*T,ydim*T)
    temp = Aexpdiagonal.dot(C_big.T)

    ddL = np.dot(C_big, temp) + K_bigInv

    return ddL



In [ ]:
def laplace(experiment,
            params,
            prevOptimRes = None,
            returnOptimRes = True,
            verbose = False,
            optimMethod = 'Newton-CG'):
    '''
    laplaceInfRes, -post_lik = laplace(experiment, params)
    '''
    ridge = 0
    
    #here T is the number of bins ydim is the number of neurons
    [ydim,T] = np.shape(experiment.data[0]['Y'])
    [ydim, xdim] = np.shape(params['C'])     #xdim is the number of latent dimensions
    numTrials = len(experiment.data)  # number of trials
    trialDur = experiment.trialDur 
    binSize = experiment.binSize      # size of bins

    # make big parameters
    C_big, d_big = util.makeCd_big(params,T)
    K_big, K = util.makeK_big(params, trialDur, binSize)
    K_bigInv = np.linalg.inv(K_big)
    
    x_post_mean = []
    x_post_cov = []
    x_vsmGP = []
    x_vsm = []

    post_lik = 0
    
    # store current optimization result to use as initialization for inference in next EM iteration
    lapOptimRes = []

    for trial in range(numTrials):
        if verbose: print('laplace inference trajectory of trial ' +str(trial+1) +'...')
        y = experiment.data[trial]['Y']
        ybar = np.ndarray.flatten(np.reshape(y, ydim*T))

        if prevOptimRes == None:
            xInit = np.ndarray.flatten(np.zeros([xdim*T,1]))
        else:
            xInit = prevOptimRes[trial]


            
        ####
        resLap = op.minimize(
            fun = negLogPosteriorUnNorm,
            x0 = xInit,
            method=optimMethod,
            args = (ybar, C_big, d_big, K_bigInv, xdim, ydim),
            jac = negLogPosteriorUnNorm_grad,
            hess = negLogPosteriorUnNorm_hess,
            options = {'disp': False,'maxiter': 10000})
        lapOptimRes.append(resLap.x)
        post_lik = post_lik + resLap.fun
        x_post_mean.append(np.reshape(resLap.x,[xdim,T]))
        hess = negLogPosteriorUnNorm_hess(resLap.x, ybar, C_big, d_big, K_bigInv, xdim, ydim)
        PostCovGP = np.linalg.inv(hess)
        
        PostCovGP = PostCovGP + ridge*np.diag(np.ones(xdim*T))
        x_post_cov.append(PostCovGP)

        temp_vsmGP = np.zeros([T,T,xdim])
        for kk in range(xdim):
            temp_vsmGP[:,:,kk] = PostCovGP[kk*T:(kk+1)*T, kk*T:(kk+1)*T]
        x_vsmGP.append(temp_vsmGP)

        temp_vsm = np.zeros([T,xdim,xdim])
        for kk in range(T):
            temp_vsm[kk][:,:] = PostCovGP[kk::T,kk::T]
        x_vsm.append(temp_vsm)
        # pdb.set_trace()

    post_lik = post_lik / numTrials
    laplaceInfRes = {
        'post_mean': x_post_mean,
        'post_cov' : x_post_cov,
        'post_vsm': x_vsm,
        'post_vsmGP': x_vsmGP}

    if returnOptimRes == True:
        return laplaceInfRes, -post_lik, lapOptimRes
    else:
        return laplaceInfRes, -post_lik


In [ ]:
def makeCd_big(params, T):
    #Here T is the number of bins in the trial
    C_big = np.kron(params['C'],np.eye(T)).T
    d_big = np.kron(np.ndarray.flatten(params['d']),np.ones(T)).T
return C_big, d_big

In [7]:
def makeK_big(params, trialDur, binSize, epsNoise = 0.001):
    [ydim,xdim] = np.shape(params['C'])
    epsSignal = 1 - epsNoise
    params['tau'] = np.ndarray.flatten(params['tau'])

    T = range(0,int(trialDur/binSize))
    K = np.zeros([xdim, len(T), len(T)])
    K_big = np.zeros([xdim*len(T), xdim*len(T)])
    
    # Make small K (size TxT) for each xdim
    for xd in range(xdim):
        for i in T:
            for j in T:
                K[xd,i,j] = epsSignal*np.exp(-0.5*((T[i]*binSize-T[j]*binSize)**2/(params['tau'][xd]*1000)**2))
        K[xd] = K[xd] + epsNoise * np.eye(len(T))

    # Make big K
    for xd in range(xdim):
        K_big[xd*len(T):(xd+1)*len(T),xd*len(T):(xd+1)*len(T)] = K[xd]
    
    return K_big, K

In [ ]:
    """Parameters:
    ===========
      * xdim       : int, latent dimensionality to fit
      * ydim       : int, number of neurons in the dataset
      * experiment : (optional) If a third optional argument of util.dataset object is given, 
                     the fucntion returns a dictionary of parameters obtained by performing Poisson-
                     PCA Leave this argument empty to initialize randomly.
    Returns:
    ========
         A dictionary of model parameters.
    """
if experiment == None:
    print('Initializing parameters randomly..')
    params = {
        'C': np.random.rand(ydim,xdim)*2 - 1,
        'd': np.random.randn(ydim)*2 - 2,
        'tau': np.random.rand(xdim)*0.5} # seconds